In [16]:
%matplotlib inline
%autosave 0
import pandas as pd
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
import plotly.express as px

Autosave disabled


In [17]:
eiaLogo = "https://upload.wikimedia.org/wikipedia/commons/1/1f/Energy_Information_Administration_logo.svg"
eiaLg = [dict(source = eiaLogo,
                       x=1.07, y=-0.1,
                       sizex=0.2, sizey=0.2,
                       xanchor="center", yanchor="bottom")]
eiaLg2 = [dict(source = eiaLogo,
                        x=1.03, y=1.1,
                        sizex=0.1, sizey=0.1,
                        xanchor="center", yanchor="bottom")]

In [2]:
rs = [('https://www.eia.gov/opendata/qb.php?sdid=NG.N9070US2.M', 'dryProduction')
     #,('https://www.eia.gov/opendata/qb.php?sdid=NG.N5060US2.M', 'stgWithdrawals')     
     ,('https://www.eia.gov/opendata/qb.php?sdid=NG.N9102US2.M', 'pplImports')
     ,('https://www.eia.gov/opendata/qb.php?sdid=NG.N9103US2.M', 'lngImports')]

storageSeason = { 1: 'wd', 2: 'wd', 3: 'wd', 11: 'wd', 12: 'wd',
                  4: 'ij', 5: 'ij', 6: 'ij', 7: 'ij', 8: 'ij', 9: 'ij', 10: 'ij'}
sData = pd.DataFrame()

for r, name in rs:
    data = pd.read_html(r)
    sName = data[0]['Series Name'][0]
    df = data[0][['Period','Value']][:]
    df['Period']  = pd.to_datetime(df.Period, format='%Y%m', errors='coerce').dropna()
    df.columns = ['Period', name]
    df = df.set_index('Period')
    df.drop(df[df.index < '2001-01-01'].index, inplace = True)
    df['Year'] = pd.DatetimeIndex(df.index).year
    df['Month'] = pd.DatetimeIndex(df.index).month
    df['sYear']  = df['Year'].shift(-3).fillna(0).astype(int).astype(str)
    df['Season'] = df['Month'].apply(lambda x: storageSeason[x]) + df['sYear']
    df.drop(df[df['Season'] == 'wd0'].index, inplace = True)

    sdf = df[[name,'Season']].groupby(['Season'],as_index=False)[name].sum()
    sdf = sdf.set_index('Season')
    print(len(sdf), name,'=', sName)
    if sData.empty:
        sData = sdf
    else:
        sData = sData.join(sdf)
        
sData *= 1000000    


42 dryProduction = U.S. Dry Natural Gas Production, Monthly
42 pplImports = U.S. Natural Gas Pipeline Imports, Monthly
42 lngImports = U.S. Liquefied Natural Gas Imports, Monthly


In [29]:
fig = px.bar(sData,  title="U.S. NG supply by storage sseasons")
fig.update_layout(xaxis_title='storage season year', yaxis_title='supply in cubic feet',legend={'traceorder': 'reversed'})
fig.layout.images = eiaLg
fig.show()

In [20]:
ijSeason = sData[sData.index.str.contains('ij')]
wdSeason = sData[sData.index.str.contains('wd')]

In [21]:
fig = px.bar(ijSeason, title="U.S. NG supply during injection seasons")
fig.update_layout(xaxis_title='injection season year', yaxis_title='supply in cubic feet',legend={'traceorder': 'reversed'})
fig.layout.images = eiaLg
fig.show()

In [24]:
fig = px.bar(wdSeason, title="U.S. NG supply during withdrawal seasons")
fig.update_layout(xaxis_title='injection season year', yaxis_title='supply in cubic feet',legend={'traceorder': 'reversed'})
fig.layout.images = eiaLg
fig.show()

In [25]:
ijSeasonAll = ijSeason.sum(axis=1)
wdSeasonAll = wdSeason.sum(axis=1)

In [26]:
fig = px.bar(x = ijSeasonAll.index, y = ijSeasonAll,
             title="U.S. NG supply during injection seasons",
             labels={'y': "supply in cubic feet", 'x': ' injection season year'})
fig.layout.showlegend = True  
fig.layout.images = eiaLg2
fig.show()   #fig.update_layout(autosize=False,width=1400,height=500,)

In [30]:
fig = px.bar(x = ijSeasonAll.index, y = ijSeasonAll.diff(), 
             title="Y/Y change in NG supply during injection seasons", 
             labels={'y': "supply in cubic feet", 'x': ' withdrawal season & year'})
             
fig.layout.showlegend = False
fig.layout.images = eiaLg2
fig.show()

In [32]:
fig = px.bar(x = wdSeasonAll.index, y = wdSeasonAll,
             title="U.S. NG supply during withdrawal seasons",
             labels={'y': "supply in cubic feet", 'x': ' withdrawal season year'})
            
fig.layout.showlegend = False
fig.layout.images = eiaLg2
fig.show()

In [33]:
fig = px.bar(x = wdSeasonAll.index, y = wdSeasonAll.diff(), 
             title="Y/Y change in NG supply during withdrawal seasons", 
             labels={'y': "supply in cubic feet", 'x': ' withdrawal season & year'})
             
fig.layout.showlegend = False
fig.layout.images = eiaLg2
fig.show()